# Neuronale Netze

## Quelle der Daten

https://www.kaggle.com/datasets/captainozlem/framingham-chd-preprocessed-data (zuletzt aufgerufen: 01/2024)

https://www.framinghamheartstudy.org/ (zuletzt aufgerufen: 01/2024)

## Installation der Bibliotheken

In [ ]:
# Importieren der benötigten Bibliotheken
import pandas as pd
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
import tensorflow as tf

## Einlesen der Daten

In [ ]:
# Definieren der URL der Datenquelle und Laden der Daten in einen pandas DataFrame
data_url = "https://github.com/timwgnd/Lehrbuch-Kuenstliche-Intelligenz-in-der-Medizin/raw/refs/heads/main/FraminghamHeartStudy.xlsx"
data = pd.read_excel(io=data_url, sheet_name = "Tabelle1")

# Entfernen von Zeilen mit fehlenden Werten
data = data.dropna()

# Anzeigen der ersten Zeilen des DataFrames
print(data.head().to_markdown(index=False, tablefmt='psql'))

In [ ]:
# Erstellen von Dummy-Variablen für die kategoriale Spalte "Geschlecht"
data_new = pd.get_dummies(data, columns = ["Geschlecht"])

# Anzeigen der ersten Zeilen des neuen DataFrames
print(data_new.head().to_markdown(index=False, tablefmt='psql'))

In [ ]:
# Zählen der Häufigkeit der einzelnen Werte in der Spalte "Diabetes" und Anzeigen in Tabellenform
count_diabetes = data_new["Diabetes"].value_counts()
print(count_diabetes.to_markdown(tablefmt='psql'))

# Erstellen eines Balkendiagramms zur Visualisierung der Verteilung
count_diabetes.plot(kind = "bar", rot = 0)

## Aufteilung der Daten und Resampling

In [ ]:
# Aufteilen der Daten in Features (x) und Zielvariable (y)
x = data_new.iloc[:, [1, 2, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14]]
y = data_new.iloc[:, 7]

In [ ]:
# Anzeigen der ersten Zeilen von x
print(x.head().to_markdown(index=False, tablefmt='psql'))

In [ ]:
# Anzeigen der ersten Zeilen von y
print(y.head().to_markdown(index=False, tablefmt='psql'))

In [ ]:
# Aufteilen der Daten in Trainings- und Testsets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.15)

In [ ]:
# Anwenden von SMOTE zur Überabtastung der Trainingsdaten
sm = SMOTE()
x_train_res, y_train_res = sm.fit_resample(x_train, y_train)

# Anzeigen der Verteilung der Zielvariable nach SMOTE
print(y_train_res.value_counts())

## FCNN-Architektur

In [ ]:
# Definieren der FCNN-Modellarchitektur
model = tf.keras.models.Sequential()

# Hinzufügen von Dense-Layern mit ReLU-Aktivierung
model.add(tf.keras.layers.Dense(128, activation = tf.nn.relu))
model.add(tf.keras.layers.Dense(256, activation = tf.nn.relu))
model.add(tf.keras.layers.Dense(128, activation = tf.nn.relu))

# Hinzufügen des Ausgabelayers mit Sigmoid-Aktivierung für binäre Klassifizierung
model.add(tf.keras.layers.Dense(1, activation = tf.nn.sigmoid))

## Modelltraining und -evaluation

In [ ]:
# Kompilieren des Modells mit Adam-Optimizer und binärer Kreuzentropie als Loss-Funktion
model.compile(optimizer = "adam",
              loss = "binary_crossentropy",
              metrics = ["accuracy"])

In [ ]:
"""
tf.keras.optimizers.Adam(
    learning_rate = 0.001,
    beta_1 = 0.9,
    beta_2 = 0.999,
    epsilon = 1e-07,
    amsgrad = False,
    name = "adam")

tf.keras.losses.BinaryCrossentropy(
    from_logits = False,
    label_smoothing = 0.0,
    axis = -1,
    name = "binary_crossentropy")

tf.keras.metrics.Accuracy(
    name = "accuracy")
"""

In [ ]:
# Trainieren des Modells
model.fit(x_train_res, y_train_res, epochs = 10)

In [ ]:
# Evaluieren des Modells auf den Testdaten
eval_results = model.evaluate(x_test, y_test)
print("[test loss, test accuracy]:", eval_results)